In [2]:
# 크롤링에 필요한 패키지 모듈 불러오기 
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

from fake_useragent import UserAgent
import random

# pass word 입력값을 감추기 위한 패키지
import getpass

import os 
import requests
import shutil
import urllib
from urllib.request import urlopen
from urllib.parse import quote_plus

# 작업 진행상황 표시를 위한 패키지 모듈 불러오기 
from tqdm import tqdm
import time
import datetime

# 데이터 판다스로 정리하기 위해 판다스 패키지 불러오기 
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

In [7]:
# 랜덤으로 생성한 UserAgent 값 출력 
ua = UserAgent()
userAgent = ua.random
print(userAgent)

Mozilla/5.0 (X11; CrOS i686 4319.74.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.57 Safari/537.36


In [10]:
# chrome dirver 옵션
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={userAgent}')  # fake user 표시
# options.add_argument('headless')                 # 크롬브라우저 숨기기
options.add_argument('window-size=1920,1080')    # 모니터 해상도 지정
options.add_argument('disable-gpu')              # CPU 부담을 줄이고 GPU로 그래픽 가속

In [11]:
# 유투브 접속, 검색 
driver = webdriver.Chrome(executable_path='/Users/kisung.Kim/Documents/chromedriver', chrome_options=options)
url = 'https://www.youtube.com/'
driver.get(url)
time.sleep(3)
driver.find_element_by_xpath('//*[@id="return-to-youtube"]').click() # 버전호환 문제로 최신 업데이트 팝업이 뜸, '나중에' 버튼 클릭 
time.sleep(3)
keyword = input("키워드를 입력해주세요: ")
url = "https://www.youtube.com/results?search_query=" + quote_plus(keyword)  # 검색한 키워드 아스키 코드로 바꾼다음 해당 페이지로 이동
driver.get(url)
time.sleep(3)

키워드를 입력해주세요: 여드름


In [12]:
# 스크롤바 내리기 
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    time_sleep = random.randint(1, 8)
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(time_sleep)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    time_sleep = random.randint(2, 9)
    if new_page_height == last_page_height:
        time.sleep(time_sleep)
        if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
            break
    else:
        last_page_height = new_page_height

In [13]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'lxml')

In [14]:
elem = soup.find_all("ytd-video-renderer", attrs={"class":'style-scope ytd-item-section-renderer'})

# 필요한 정보 가져오기
df = []
for t in elem:
    title = t.find("yt-formatted-string", attrs={"class":'style-scope ytd-video-renderer'}).get_text()
    name = t.find("a", attrs={"class":'yt-simple-endpoint style-scope yt-formatted-string'}).get_text()
    thumbnail = t.find('img', attrs={'class':'style-scope yt-img-shadow'}).get('src')
    content_url = t.find("a", attrs={"class":'yt-simple-endpoint style-scope ytd-video-renderer'})["href"]
    df.append([name, title , thumbnail,  'https://www.youtube.com/'+content_url])

In [16]:
driver.close()

In [20]:
df

[['여드름닥터',
  '속에서 만져지기만 하고 짤수도 없는 여드름은 이렇게 해결하세요(여드름 없애는 법)',
  'https://i.ytimg.com/vi/v4IKX6habfM/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDGZwM1eSO4nOq8HQluRHLuKJHKVA',
  'https://www.youtube.com//watch?v=v4IKX6habfM'],
 ['피부왕정윤',
  '피부과전문의가 말하는 여드름 좋아지는 홈케어 비결 | 피부좋아지는법',
  'https://i.ytimg.com/vi/iSTRRTGItAw/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCmmg0sXh6Mt3RHMFYl9dnMKoLsYw',
  'https://www.youtube.com//watch?v=iSTRRTGItAw'],
 ['마이린 TV',
  '[일상] 병원에서 여드름 레이저 치료를 받았어요 ㅠㅠ (피부과 상담부터 여드름 케어, 포렌자 니들 RF 치료까지 풀 중계 브이로그) | 마이린 TV',
  'https://i.ytimg.com/vi/85m6Qidw54E/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDktfMVOM4MOsUYO-AS93G9uWSGGg',
  'https://www.youtube.com//watch?v=85m6Qidw54E'],
 ['뷰드름 유튜버 인씨',
  '29년 여드름 피부가 절대 안하는 것 3가지',
  'https://i.ytimg.com/vi/WLPdIx7NoD0/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLAH3iK3QYutkk6n-hz1m698_yD1Hw',
  'https://www.yo

In [22]:
df_list = pd.DataFrame(columns=['title', 'name', 'thumbnail_link', 'url_link'])
# 자료 넣기 
for i in range(len(df)):
    df_list.loc[i] = df[i]
    
# 저장 폴더 
# 폴더 생성
# 폴더의 존재유무파악 (같은 이름 X)
# os: 파일 경로를 찾아줌
if not os.path.isdir('/Users/kisung.Kim/Documents/kimkisung/Work/Dermal_Freeco_Project/img_data_collection/youtube_img'):
    print('폴더 생성 완료')
    os.mkdir('/Users/kisung.Kim/Documents/kimkisung/Work/Dermal_Freeco_Project/img_data_collection/youtube_img')
else:
    print('이미 동일한 이름의 폴더가 있습니다.')

now = datetime.datetime.now()
now = str(now).replace(' ','')

df_dir = "./youtube_img/"
df_list.to_csv(df_dir + f"youtube{keyword}_{now}_df.csv")

이미 동일한 이름의 폴더가 있습니다.


In [21]:
# crawling_list = []
# url_list = []

# # url 수집 
# crawling_list = driver.find_elements_by_tag_name('h3 > a')

# for crawling in crawling_list:
#     url = crawling.get_attribute('href')
#     url_list.append(url)


# # url 리스트 출력
# print("크롤링할 갯수: ", len(url_list))
# url_list

# url_df = pd.DataFrame()
# url_df['link'] = url_list

# now = datetime.datetime.now()
# now = str(now).replace(' ','')

# url_df.to_csv('./img/' + f'키워드_{keyword}youtube{now}.csv', index=False)
# driver.close()

크롤링할 갯수:  77


['https://www.youtube.com/watch?v=v4IKX6habfM',
 'https://www.youtube.com/watch?v=whbpLnYOJ84',
 'https://www.youtube.com/watch?v=iSTRRTGItAw',
 'https://www.youtube.com/watch?v=WLPdIx7NoD0',
 'https://www.youtube.com/watch?v=a-fRpsCBD1w',
 'https://www.youtube.com/watch?v=85m6Qidw54E',
 'https://www.youtube.com/watch?v=DtKsokgyHOY',
 'https://www.youtube.com/watch?v=4YlWKAywpBw',
 'https://www.youtube.com/watch?v=xQzKbSq-euI',
 'https://www.youtube.com/watch?v=U6JstVPus5Q',
 'https://www.youtube.com/watch?v=xDNEieOQHiY',
 'https://www.youtube.com/watch?v=BV4nLxkaXgQ',
 'https://www.youtube.com/watch?v=eogEUk3FpCc',
 'https://www.youtube.com/watch?v=gybsN3UDs-8',
 'https://www.youtube.com/watch?v=wAT8MGuFIWs',
 'https://www.youtube.com/watch?v=Hwts65ISf9c',
 'https://www.youtube.com/watch?v=Hn-Ruweh6gU',
 'https://www.youtube.com/watch?v=Zr94ZpgyvXw',
 'https://www.youtube.com/watch?v=WS6OEN4bhPI',
 'https://www.youtube.com/watch?v=SdDDmQ1YOs4',
 'https://www.youtube.com/watch?v=rP2TEe